# MLG HW2

In [1]:
import pandas as pd
import numpy as np
import os, time, torch, json
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
# from torch_geometric.utils import accuracy,sparse_mx_to_torch_sparse_tensor
import torch_geometric.utils 
# from models.GCN import GCN
import scipy.sparse as sp
from tqdm import tqdm, trange
from torch.autograd import Variable
import torchvision
from torch.utils.data import DataLoader, TensorDataset
# from torchvision import datasets, transforms
# import torch.utils.data as Data
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from scipy.stats import entropy


import os.path as osp

import torch
from sklearn.metrics import roc_auc_score

import torch_geometric.transforms as T
from torch_geometric.datasets import Planetoid
from torch_geometric.nn import GCNConv
from torch_geometric.utils import negative_sampling
from torch_geometric.data import Data, Dataset
# test 
from sklearn.metrics import roc_auc_score, average_precision_score

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)
%autosave 120

/home/rita/miniconda3/envs/jupyterlab/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


cuda


Autosaving every 120 seconds


In [2]:
content = []
test_ = []
train_ = []
upload = []
edge_index = [] 
for i in range(3):
    os.chdir('/home/rita/111/111-2MLG/HW2/dataset{}'.format(i + 1))
    temp = pd.read_csv('./content.csv', header = None, sep = '\t')
    temp.sort_values(by = [0], inplace = True)
    temp.set_index([0], inplace = True)
    # temp = torch.Tensor(np.array(temp)).to(torch.float32)
    # print(temp.shape)
    content.append(temp)
    test_.append(pd.read_csv('./test.csv'))
    temp = pd.read_csv('./train.csv')
    # print(temp.shape)
    train_.append(temp)
    temp = temp[temp.label == 1]
    temp = temp[['to', 'from']]
    temp = temp.reset_index(drop = True)
    # print(temp.shape)
    edge_index.append(temp)
    upload.append(pd.read_csv('./upload.csv'))
print('content :\n', content[0][:2])
print('test_ :\n', test_[0].head(2))
print('train_ : \n', train_[0].head(2))
print('edge_index : \n', edge_index[0].head(2))
print('upload : \n', upload[0].head(2))
os.chdir('/home/rita/111/111-2MLG/HW2')

content :
    1     2     3     4     5     6     7     8     9     10    ...  1424  \
0                                                              ...         
0     0     0     0     0     0     0     0     0     0     0  ...     0   
1     0     0     0     0     0     0     0     0     0     0  ...     0   

   1425  1426  1427  1428  1429  1430  1431  1432  1433  
0                                                        
0     0     0     0     0     0     0     0     0     0  
1     1     0     0     0     0     0     0     0     0  

[2 rows x 1433 columns]
test_ :
        id    to  from
0  E10559  2323  2673
1   E4849    81  1634
train_ : 
        id    to  from  label
0  E10311  2399  2339      0
1  E10255  2397  1144      1
edge_index : 
      to  from
0  2397  1144
1  2450  1312
upload : 
        id  prob
0  E10559   0.5
1   E4849   0.5


In [3]:
# content preprocessing
new_features = []
for j in range(len(content)) :
    t = torch.Tensor(np.array(content[j]))
    features_entropy = []
    for i in range(t.shape[1]) :
        temp = t.T[i]
        t1 = torch.sum(temp == 0) / len(temp)
        t2 = torch.sum(temp == 1) / len(temp)
        temp = torch.tensor([t1, t2])
        temp = entropy(temp)
        if (temp == 0) :
            temp = 0
        else :
            temp = 1 / temp
        features_entropy.append(temp)
    features_entropy = torch.tensor(features_entropy).reshape(1, -1)
    t = t * features_entropy
    t = t.type(torch.float32)
    new_features.append(t)
print(new_features[0][:2])

tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]])


In [21]:
# change to Data.Data

# data = {
#     'num_features' : content[i].shape[1], 
#     'num_nodes' : content[i].shape[0], 
#     'x' : content[i], 
#     'edge_index' : torch.Tensor(np.array(train_[i].iloc[::, 1:3])).T.type(torch.long), 
#     'edge_label_index' : torch.Tensor(np.array(train_[i][train_[i].label == 1].iloc[::, 1:3])).T.type(torch.long), 
#     'edge_label' : torch.Tensor(np.array(train_[i][train_[i].label == 1].iloc[::, -1]))
# }

train_pyg = {}
test_pyg = {}
for i in range(3):
    x = torch.Tensor(np.array(content[i]))
    # x = new_features[i]
    train_edge_index = torch.Tensor(np.array(train_[i].iloc[::, 1:3])).T.type(torch.long)
    train_edge_label_index = torch.Tensor(np.array(train_[i][train_[i].label == 1].iloc[::, 1:3])).T.type(torch.long)
    train_edge_label = torch.Tensor(np.array(train_[i][train_[i].label == 1].iloc[::, -1]))
    test_edge_label_index = torch.Tensor(np.array(test_[i].iloc[::, 1:3]).astype(float)).T.type(torch.long)
    train_data = Data(x = x, edge_index = train_edge_index, edge_label_index = train_edge_label_index, edge_label = train_edge_label)
    test_data = Data(x = x, edge_index = train_edge_index, edge_label_index = test_edge_label_index)

    train_pyg[i] = train_data.to(device)
    test_pyg[i] = test_data.to(device)
print(train_pyg[2])
print(test_pyg[2])


Data(x=[877, 1703], edge_index=[2, 2572], edge_label_index=[2, 1273], edge_label=[1273])
Data(x=[877, 1703], edge_index=[2, 2572], edge_label_index=[2, 644])


In [22]:
transform = T.Compose([
    T.NormalizeFeatures(),
    T.ToDevice(device),
    # T.RandomLinkSplit(num_val=0.05, num_test=0.1, is_undirected=True,
    #                   add_negative_train_samples=False),
])
# CUDA_LAUNCH_BLOCKING=1 
for i in range(3) :
    train_pyg[i] = transform(train_pyg[i])
    test_pyg[i] = transform(test_pyg[i])

## Cora Test

In [6]:
dataset = Planetoid("/tmp/Cora", name = "Cora")
num_nodes = dataset.data.num_nodes 
# For num. edges see: 
# - https://github.com/pyg-team/pytorch_geometric/issues/343 
# - https://github.com/pyg-team/pytorch_geometric/issues/852 
num_edges = dataset.data.num_edges // 2 
train_len = dataset[0].train_mask.sum() 
val_len = dataset[0].val_mask.sum() 
test_len = dataset[0].test_mask.sum() 
other_len = num_nodes - train_len - val_len - test_len 
print(f"Dataset: {dataset.name}") 
print(f"Num. nodes: {num_nodes} (train={train_len}, val={val_len}, test={test_len}, other={other_len})") 
print(f"Num. edges: {num_edges}") 
print(f"Num. node features: {dataset.num_node_features}") 
print(f"Num. classes: {dataset.num_classes}") 
print(f"Dataset len.: {dataset.len()}")

dataset = Planetoid("/tmp/Cora", name="Cora") 
print(f"Sum of row values without normalization: {dataset[0].x.sum(dim=-1)}") 
 
dataset = Planetoid("/tmp/Cora", name="Cora", transform=T.NormalizeFeatures()) 
print(f"Sum of row values with normalization: {dataset[0].x.sum(dim=-1)}")

print(dataset[0])
print(dataset)
print(dataset[0].keys)

Dataset: Cora
Num. nodes: 2708 (train=140, val=500, test=1000, other=1068)
Num. edges: 5278
Num. node features: 1433
Num. classes: 7
Dataset len.: 1
Sum of row values without normalization: tensor([ 9., 23., 19.,  ..., 18., 14., 13.])
Sum of row values with normalization: tensor([1.0000, 1.0000, 1.0000,  ..., 1.0000, 1.0000, 1.0000])
Data(x=[2708, 1433], edge_index=[2, 10556], y=[2708], train_mask=[2708], val_mask=[2708], test_mask=[2708])
Cora()
['edge_index', 'test_mask', 'x', 'train_mask', 'val_mask', 'y']


In [7]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
transform = T.Compose([
    T.NormalizeFeatures(),
    T.ToDevice(device),
    T.RandomLinkSplit(num_val=0.05, num_test=0.1, is_undirected=True,
                      add_negative_train_samples=False),
])
# path = osp.join(osp.dirname(osp.realpath(__file__)), '..', 'data', 'Planetoid')
# dataset = Planetoid(path, name='Cora', transform=transform)
dataset = Planetoid("/tmp/Cora", name = "Cora", transform=transform)
# After applying the `RandomLinkSplit` transform, the data is transformed from
# a data object to a list of tuples (train_data, val_data, test_data), with
# each element representing the corresponding split.
train_data, val_data, test_data = dataset[0]

# print(train_data)
# print(val_data)
# print(test_data)
print(dataset[0])

(Data(x=[2708, 1433], edge_index=[2, 8976], y=[2708], train_mask=[2708], val_mask=[2708], test_mask=[2708], edge_label=[4488], edge_label_index=[2, 4488]), Data(x=[2708, 1433], edge_index=[2, 8976], y=[2708], train_mask=[2708], val_mask=[2708], test_mask=[2708], edge_label=[526], edge_label_index=[2, 526]), Data(x=[2708, 1433], edge_index=[2, 9502], y=[2708], train_mask=[2708], val_mask=[2708], test_mask=[2708], edge_label=[1054], edge_label_index=[2, 1054]))


In [ ]:
print('dataset.num_features :', dataset.num_features)
print('train_data.x : ', train_data.x)
# 所有的 edge (0, 1)
print('train_data.edge_index : ', train_data.edge_index.shape)
print('train_data.edge_index : ', train_data.edge_index)
# label = 1 的 edge
print('train_data.edge_label_index : ', train_data.edge_label_index.shape)
print('train_data.edge_label_index : ', train_data.edge_label_index)
print('train_data.edge_label_index.size(1) : ', train_data.edge_label_index.size(1))
print('train_data.edge_label : ', train_data.edge_label.shape)
print('train_data.edge_label : ', train_data.edge_label)

In [ ]:
class Net(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels):
        super().__init__()
        self.conv1 = GCNConv(in_channels, hidden_channels)
        self.conv2 = GCNConv(hidden_channels, out_channels)

    def encode(self, x, edge_index):
        x = self.conv1(x, edge_index).relu()
        return self.conv2(x, edge_index)

    def decode(self, z, edge_label_index):
        return (z[edge_label_index[0]] * z[edge_label_index[1]]).sum(dim=-1)

    def decode_all(self, z):
        prob_adj = z @ z.t()
        return (prob_adj > 0).nonzero(as_tuple=False).t()


model = Net(dataset.num_features, 128, 64).to(device)
optimizer = torch.optim.Adam(params=model.parameters(), lr=0.01)
criterion = torch.nn.BCEWithLogitsLoss()


def train():
    model.train()
    optimizer.zero_grad()
    z = model.encode(train_data.x, train_data.edge_index)

    # We perform a new round of negative sampling for every training epoch:
    neg_edge_index = negative_sampling(
        edge_index=train_data.edge_index, num_nodes=train_data.num_nodes,
        num_neg_samples=train_data.edge_label_index.size(1), method='sparse')

    edge_label_index = torch.cat(
        [train_data.edge_label_index, neg_edge_index],
        dim=-1,
    )
    edge_label = torch.cat([
        train_data.edge_label,
        train_data.edge_label.new_zeros(neg_edge_index.size(1))
    ], dim=0)

    out = model.decode(z, edge_label_index).view(-1)
    loss = criterion(out, edge_label)
    loss.backward()
    optimizer.step()
    return loss


@torch.no_grad()
def test(data):
    model.eval()
    z = model.encode(data.x, data.edge_index)
    out = model.decode(z, data.edge_label_index).view(-1).sigmoid()
    return roc_auc_score(data.edge_label.cpu().numpy(), out.cpu().numpy())


best_val_auc = final_test_auc = 0
for epoch in range(1, 101):
    loss = train()
    val_auc = test(val_data)
    test_auc = test(test_data)
    if val_auc > best_val_auc:
        best_val_auc = val_auc
        final_test_auc = test_auc
    print(f'Epoch: {epoch:03d}, Loss: {loss:.4f}, Val: {val_auc:.4f}, '
          f'Test: {test_auc:.4f}')

print(f'Final Test: {final_test_auc:.4f}')

z = model.encode(test_data.x, test_data.edge_index)
final_edge_index = model.decode_all(z)

## For HW

In [23]:
class Net(nn.Module):
    def __init__(self, in_channels, hidden_channels = 128, out_channels = 64):
        super().__init__()
        self.conv1 = GCNConv(in_channels, hidden_channels)
        self.conv2 = GCNConv(hidden_channels, out_channels)

    def encode(self, x, edge_index):
        x = nn.functional.leaky_relu(self.conv1(x, edge_index))
        # x = self.conv1(x, edge_index).relu()
        return self.conv2(x, edge_index)

    def decode(self, z, edge_label_index):
        return (z[edge_label_index[0]] * z[edge_label_index[1]]).sum(dim=-1)

    def decode_all(self, z):
        prob_adj = z @ z.t()
        return (prob_adj > 0).nonzero(as_tuple=False).t()

def fit(model, optim, loss_fn, train_data):
    model.train()
    optim.zero_grad()
    z = model.encode(train_data.x, train_data.edge_index)
    # z = model.encode(data['x'].to(device), data['edge_index'].to(device))

    # We perform a new round of negative sampling for every training epoch:
    neg_edge_index = negative_sampling(
        edge_index=train_data.edge_index, num_nodes=train_data.num_nodes,
        num_neg_samples=train_data.edge_label_index.size(1), method='sparse')

    edge_label_index = torch.cat(
        [train_data.edge_label_index, neg_edge_index],
        dim=-1,
    )
    edge_label = torch.cat([
        train_data.edge_label,
        train_data.edge_label.new_zeros(neg_edge_index.size(1))
    ], dim=0)
    # neg_edge_index = negative_sampling(
    #     edge_index= data['edge_index'], num_nodes= data['num_nodes'],
    #     num_neg_samples=data['edge_label_index'].size(1), method='sparse')

    # edge_label_index = torch.cat(
    #     [data['edge_label_index'], neg_edge_index],
    #     dim=-1,
    # )
    # edge_label = torch.cat([
    #     data['edge_label'],
    #     data['edge_label'].new_zeros(neg_edge_index.size(1))
    # ], dim=0)

    out = model.decode(z, edge_label_index).view(-1)
    loss = loss_fn(out, edge_label.to(device))
    loss.backward()
    optim.step()
    return loss


# @torch.no_grad()
# def test(data):
#     model.eval()
#     z = model.encode(data['x'], data['edge_index'])
#     out = model.decode(z, data['edge_label_index']).view(-1).sigmoid()
#     return roc_auc_score(data['edge_label'].cpu().numpy(), out.cpu().numpy())

def training_loop(data, n_epochs):
    model = Net(in_channels = data.x.shape[1]).to(device)
    optim = torch.optim.Adam(params=model.parameters(), lr=1e-2)
    loss_fn = torch.nn.BCEWithLogitsLoss()
    for epoch in range(1, n_epochs + 1):
        loss = fit(
            model = model, 
            optim = optim, 
            loss_fn = loss_fn, 
            train_data = data
        )
        if epoch % 100 == 0 :
            print('Epoch : {}, Loss : {}'.format(epoch, loss))
    
    return model

# model = Net(dataset.num_features, 128, 64).to(device)
# model = Net(content[i].shape[1], 128, 64).to(device)
# # optimizer = torch.optim.Adam(params=model.parameters(), lr=0.01)
# optimizer = torch.optim.Adam(params=model.parameters(), lr=1e-3)
# criterion = torch.nn.BCEWithLogitsLoss()

# best_val_auc = final_test_auc = 0
# for epoch in range(1, 11):
#     loss = train(data = data)
# #     val_auc = test(val_data)
# #     test_auc = test(test_data)
# #     if val_auc > best_val_auc:
# #         best_val_auc = val_auc
# #         final_test_auc = test_auc
# #     print(f'Epoch: {epoch:03d}, Loss: {loss:.4f}, Val: {val_auc:.4f}, '
# #           f'Test: {test_auc:.4f}')
#     print(f'Epoch: {epoch:03d}, Loss: {loss:.4f}')

# print(f'Final Test: {final_test_auc:.4f}')

# z = model.encode(test_data.x, test_data.edge_index)
# final_edge_index = model.decode_all(z)

In [24]:
def predict(model, test) :
    z = model.encode(test.x, test.edge_index)
    pred = torch.sigmoid(model.decode(z, test.edge_label_index))
    

    # test_x = torch.Tensor(np.array(test.iloc[::, 1:])).to(device)
    # test_y = loaded_model(test_x)
    # test_y = pd.DataFrame(test_y)
    # pred = pd.concat([test, test_y], axis = 1)
    # pred = pred.drop(['to', 'from'], axis = 1)
    # pred.columns = ['id', 'prob']
    return pred

In [25]:
for i in range(3):
    if i == 2 :
        n_epochs = 10000
    else :
        n_epochs = 5000

    model = training_loop(data = train_pyg[i], n_epochs = n_epochs)
    # pred = predict(model, test_pyg[i])

    z = model.encode(test_pyg[i].x, test_pyg[i].edge_index)
    pred = torch.sigmoid(model.decode(z, test_pyg[i].edge_label_index))
    print(pred.shape)
    pred = pd.DataFrame(pred.cpu().detach().numpy())
    pred = pd.concat([test_[i], pred], axis = 1)
    pred = pred.drop(['to', 'from'], axis = 1)
    pred.columns = ['id', 'prob']
    pred.to_csv('./upload/pred_pyg_norm_leaky_{}.csv'.format(i + 1))



Epoch : 100, Loss : 0.4607372283935547
Epoch : 200, Loss : 0.44116517901420593
Epoch : 300, Loss : 0.4264395833015442
Epoch : 400, Loss : 0.4205430746078491
Epoch : 500, Loss : 0.4126579463481903
Epoch : 600, Loss : 0.4061504602432251
Epoch : 700, Loss : 0.4038366675376892
Epoch : 800, Loss : 0.40121349692344666
Epoch : 900, Loss : 0.3962206542491913
Epoch : 1000, Loss : 0.39370569586753845
Epoch : 1100, Loss : 0.3961440622806549
Epoch : 1200, Loss : 0.3943099081516266
Epoch : 1300, Loss : 0.3916400372982025
Epoch : 1400, Loss : 0.39491578936576843
Epoch : 1500, Loss : 0.3881003260612488
Epoch : 1600, Loss : 0.38706809282302856
Epoch : 1700, Loss : 0.388090044260025
Epoch : 1800, Loss : 0.3885413706302643
Epoch : 1900, Loss : 0.38455501198768616
Epoch : 2000, Loss : 0.38809698820114136
Epoch : 2100, Loss : 0.38423606753349304
Epoch : 2200, Loss : 0.3866063952445984
Epoch : 2300, Loss : 0.3830832242965698
Epoch : 2400, Loss : 0.38732248544692993
Epoch : 2500, Loss : 0.38253697752952576


In [ ]:
def predict(test_data, model):
    z = model.encode(test_data.x, test_data.edge_index)
    test_pred = torch.sigmoid(model.decode(z, test_data.edge_label_index))
    return test_pred